In [1]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install keras

In [2]:

import numpy as np
import pandas as pd
from keras.callbacks import TensorBoard
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import plot_model
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [3]:
class NNKeras:
    def __init__(self, url: str):
        self._url = url
        self._num_cols = 64
        self._call_back = TensorBoard(log_dir='../logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

    def read_data(self):
        df = pd.read_csv(self._url, header=None)
        X = df.iloc[:, 1:].astype(float)
        classes = df.iloc[:, 0]
        unique_classes = pd.DataFrame(sorted([c.upper() for c in classes.unique()]))
        rows = X.shape[0]
        unique_classes['indices'] = range(1, len(unique_classes) + 1)
        y = np.zeros((rows, len(unique_classes)), np.bool)
        for i in range(rows):
            col_idx = np.where(unique_classes.loc[:, 0] == classes[i].upper())
            y[i, col_idx] = True
        return X, y, unique_classes

    def base_model(self, nodes):
        model = Sequential()

        for prev_node, node in zip(nodes[:-1], nodes[1:]):
            model.add(Dense(node, activation='relu', input_dim=prev_node))  # Add the first hidden layer

        model.add(Dense(31, activation='sigmoid'))  # Add the output layer
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def train(self, X, y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=5)

        for num_nodes in range(2, 100):
            nodes = [64, num_nodes]
            model = self.base_model(nodes)
            model.fit(X_train, y_train, epochs=1, verbose=0, callbacks=[self._call_back])
            loss, accuracy = model.evaluate(X_test, y_test)
            print(num_nodes, accuracy)

In [4]:
nn = NNKeras("https://raw.githubusercontent.com/hardikgw/neural-network-poc/master/data/dataset/dataset.csv")

In [5]:
X, y, unique_classes = nn.read_data()

In [ ]:
nn.train(X, y)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
157/157 [==============================] - 0s 599us/step
2 0.9677418956331386
157/157 [==============================] - 0s 453us/step
3 0.9677418956331386
157/157 [==============================] - 0s 524us/step
4 0.9677418956331386
157/157 [==============================] - 0s 682us/step
5 0.9675364243756434
157/157 [==============================] - 0s 1ms/step
6 0.941236913963488
157/157 [==============================] - 0s 1ms/step
7 0.9677418956331386
157/157 [==============================] - 0s 1ms/step
8 0.9677418956331386
157/157 [==============================] - 0s 1ms/step
9 0.9677418956331386
157/157 [==============================] - 0s 2ms/step
10 0.9677418956331386
157/157 [==============================] - 0s 2ms/step
11 0.9677418956331386
157/157 [==============================] - 1s 3ms/step
12 0.967947366890634
157/157 [==========================